# Introduction

The following notebook is meant to visualise the steps taken to automatically register probes from probeinterface to histology dye stains.



In [1]:
# Setup
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from importlib import reload
from pathlib import Path
from probe_auto_registration import probeinterface_tracing as pit

In [2]:
#load data
reload(pit)
subject_ID = 'EX03'
brainreg_atlas_path = Path(f'../data/raw_data/histology/{subject_ID}/anat/allen_mouse_10um')
data = pit.get_data(brainreg_atlas_path)

## mapping coordinates to allen brain space



In [ ]:
# let's map the labels and then the coordinates as well.

threshold_signal = pit.threshold_signal_gamma(data['signal_data'])
signal_df = pit.make_signal_df(data['signal_data'], threshold_signal)
signal_df = pit.cluster_signal(signal_df, n_clusters=2)
signal_df1 = signal_df[signal_df['cluster'] == 1]
probe_df = pit.get_probe_contacts_df()
fig = go.Figure()
for cluster in [0,1]:
    signal_cluster_df = signal_df[signal_df['cluster'] == cluster]
    plane = pit.fit_plane_to_signal(signal_cluster_df)
    results = pit.optimize_probe_plane(signal_cluster_df,probe_df,plane)
    if results['result'].success:
        print(f"Fitted parameters: {results['result'].x}")
        pit.plot_3d(signal_cluster_df,results['coords'].values, fig=fig)

In [83]:
results['coords'].values

import numpy as np
import tifffile
from scipy.ndimage import map_coordinates

# ----- Step 1: Load transforms -----
affine = data['affine_matrix'] # 4×4 matrix
# Inverse affine or nonlinear outputs not used here

# Assuming brainreg’s deformation fields (3 × TIFFs)
def_field = np.stack((data['deformation_field_0'], 
                      data['deformation_field_1'], 
                      data['deformation_field_2']), axis=0)  # shape = (3, z, y, x)

def sample_deformation(points_vox):
    # points: (N,3) in (z, y, x)
    coords = [points_vox[:,0], points_vox[:,1], points_vox[:,2]]
    return np.stack([
        map_coordinates(def_field[0], coords, order=1),
        map_coordinates(def_field[1], coords, order=1),
        map_coordinates(def_field[2], coords, order=1),
    ], axis=1)

# ----- Step 2: Your input points in reoriented/downsampled space -----
input_pts =results['coords'].values

# Apply affine: convert voxel (z,y,x) → add homogeneous → x, y, z
pts_homog = np.hstack([
    input_pts[:, ::-1],  # (z,y,x) → (x,y,z)
    np.ones((len(input_pts), 1))
])
affine_out = (affine @ pts_homog.T).T  # (N,4)
affine_vox_atlas = affine_out[:, :3][:, ::-1]  # back to (z,y,x) order if needed

# ----- Step 3: Apply nonlinear warp (deformation fields) -----
disp = sample_deformation(affine_vox_atlas)  # (N,3)
# Decide: are they displacement or absolute? Assume displacement:
atlas_vox = affine_vox_atlas + disp

# Optional: convert voxel → µm using atlas voxel size (e.g., 10 µm)
atlas_um = atlas_vox * 10.0

print("Final atlas coordinates (voxels):", atlas_vox)
print("Final atlas coordinates (µm):", atlas_um)

Final atlas coordinates (voxels): [[800.2066556  473.33564397 263.86255455]
 [802.72408595 472.83096319 263.92052033]
 [799.91847553 471.80852573 264.01293261]
 ...
 [803.1894047  149.63706591 295.95562263]
 [800.38311527 148.60670604 296.04038382]
 [802.90028097 148.10206734 296.09838989]]

Final atlas coordinates (µm): [[8002.06655595 4733.3564397  2638.62554553]
 [8027.24085947 4728.30963191 2639.20520333]
 [7999.18475526 4718.08525734 2640.12932605]
 ...
 [8031.89404704 1496.37065908 2959.55622633]
 [8003.83115267 1486.0670604  2960.40383815]
 [8029.00280974 1481.02067342 2960.98389887]]

In [85]:
def sample_coords_to_allen_space(points_array:np.array, 
                                 data:dict,
                                 voxel_size:int = 10,
                                ):
    # 1. apply affine transformation
    affine = data['affine_matrix'] # 4×4 matrix
    pts_homog = np.hstack([ points_array[:, ::-1],  # (z,y,x) → (x,y,z)
                            np.ones((len(points_array), 1))])
    affine_out = (affine @ pts_homog.T).T  # (N,4)
    affine_vox_atlas = affine_out[:, :3][:, ::-1]  # back to (z,y,x) for deformation
    # 2. apply deformation field
    coords = [affine_vox_atlas[:,i] for i in range(3)]
    disp = np.stack([
        map_coordinates(data['deformation_field_0'], coords, order=1),
        map_coordinates(data['deformation_field_1'], coords, order=1),
        map_coordinates(data['deformation_field_2'], coords, order=1),
    ], axis=1)

    atlas_vox = affine_vox_atlas + disp
    atlas_um = atlas_vox * voxel_size
    return atlas_um


atlas_um = sample_coords_to_allen_space(results['coords'].values,
data)
atlas_um


array([[8002.06655595, 4733.3564397 , 2638.62554553],
       [8027.24085947, 4728.30963191, 2639.20520333],
       [7999.18475526, 4718.08525734, 2640.12932605],
       ...,
       [8031.89404704, 1496.37065908, 2959.55622633],
       [8003.83115267, 1486.0670604 , 2960.40383815],
       [8029.00280974, 1481.02067342, 2960.98389887]], shape=(1648, 3))

In [79]:
volume_ids = []
structure_names = []
acronyms = []
for each_coord in signal_df[['x','y','z']].values:
    int_coords = [int(x) for x in each_coord]
    volume_ids.append(data['atlas_registration_data'][int_coords[0],int_coords[1],int_coords[2]])
    if volume_ids[-1]< len(data['volumes_df']):
        
        structure_names.append(data['volumes_df'].iloc[volume_ids[-1]].structure_name)
        acronyms.append(ALLEN_NAMES2ACRONYM[structure_names[-1]])
    else:
        structure_names.append('outside brain')
        acronyms.append(np.nan)

np.unique(structure_names)

array(['Agranular insular area, posterior part, layer 2/3',
       'Agranular insular area, posterior part, layer 5',
       'Anterior area, layer 5',
       'Anterior cingulate area, dorsal part, layer 6b',
       'Area prostriata', 'Basolateral amygdalar nucleus, anterior part',
       'Central medial nucleus of the thalamus', 'Crus 2',
       'Cuneiform nucleus', 'Interfascicular nucleus raphe',
       'Lateral vestibular nucleus', 'Laterointermediate area, layer 4',
       'Medial mammillary nucleus, lateral part',
       'Medial mammillary nucleus, median part', 'Pontine gray',
       'Posterolateral visual area, layer 5', 'Postrhinal area, layer 4',
       'Primary auditory area, layer 5',
       'Primary auditory area, layer 6b', 'Pyramus (VIII)',
       'Retrosplenial area, lateral agranular part, layer 5',
       'Septofimbrial nucleus', 'Supraoculomotor periaqueductal gray',
       'Tuberomammillary nucleus, ventral part',
       'Ventral auditory area, layer 4', 'dorsal spin

In [18]:
atlas_structure_key = pd.read_csv('./probe_auto_registration/allen_brain_atlas_info.htsv', sep='\t', index_col=0)
data['volumes_df']

,structure_name,left_volume_mm3,right_volume_mm3,total_volume_mm3
0,"Tuberomammillary nucleus, ventral part",0.029499,0.026311,0.055810
1,"Primary somatosensory area, mouth, layer 6b",0.045815,0.043552,0.089367
2,internal capsule,0.883631,0.957353,1.840984
3,Principal sensory nucleus of the trigeminal,0.562637,0.575630,1.138267
4,"Primary somatosensory area, trunk, layer 6a",0.101296,0.054981,0.156277
...,...,...,...,...
666,"Interpeduncular nucleus, intermediate",0.017865,0.019142,0.037007
667,"Interpeduncular nucleus, dorsomedial",0.011747,0.012057,0.023804
668,"Interpeduncular nucleus, dorsolateral",0.025637,0.026798,0.052435
669,"Interpeduncular nucleus, rostrolateral",0.009801,0.010563,0.020364


In [42]:
volumes_df = data['volumes_df']
name2acronym = dict(zip(atlas_structure_key.name,atlas_structure_key.acronym))
volumes_df['acronym'] = volumes_df['structure_name'].apply(lambda x: name2acronym[x])
volumes_df

,structure_name,left_volume_mm3,right_volume_mm3,total_volume_mm3,acronym
0,"Tuberomammillary nucleus, ventral part",0.029499,0.026311,0.055810,TMv
1,"Primary somatosensory area, mouth, layer 6b",0.045815,0.043552,0.089367,SSp-m6b
2,internal capsule,0.883631,0.957353,1.840984,int
3,Principal sensory nucleus of the trigeminal,0.562637,0.575630,1.138267,PSV
4,"Primary somatosensory area, trunk, layer 6a",0.101296,0.054981,0.156277,SSp-tr6a
...,...,...,...,...,...
666,"Interpeduncular nucleus, intermediate",0.017865,0.019142,0.037007,IPI
667,"Interpeduncular nucleus, dorsomedial",0.011747,0.012057,0.023804,IPDM
668,"Interpeduncular nucleus, dorsolateral",0.025637,0.026798,0.052435,IPDL
669,"Interpeduncular nucleus, rostrolateral",0.009801,0.010563,0.020364,IPRL


In [43]:
import json

with open('./probe_auto_registration/allen_name2acronym.json', 'w') as f:
    json.dump(name2acronym, f, indent=2)

## plotting and animating with brainrender


In [ ]:

from pathlib import Path

from myterial import orange
from rich import print

from brainrender import Animation, Scene
from brainrender.actors import Points

output_path = Path("../results/brainrender/")

scene = Scene(atlas_name="allen_mouse_10um")

regions = (
    "CA1","CA3",
)
scene.add_brain_region(*regions, silhouette=True, alpha=0.3)
scene.add(Points(atlas_um,
        name="probe_1",
        colors="darkred",
        radius=50))

anim = Animation(scene, output_path, "brainrender_animation")

# Specify camera position and zoom at some key frames
# each key frame defines the scene's state after n seconds have passed
anim.add_keyframe(0, camera="top", zoom=1)
anim.add_keyframe(1.5, camera="sagittal", zoom=0.95)
anim.add_keyframe(3, camera="frontal", zoom=1)
anim.add_keyframe(4, camera="frontal", zoom=1.2)

# Make videos
anim.make_video(duration=5, fps=15, resetcam=True)

Saving video in ../results/brainrender

📽  Video file brainrender_animation is open... 

Output()

Saving video

Saved video at: ../results/brainrender/brainrender_animation.mp4

'../results/brainrender/brainrender_animation.mp4'

In [7]:
from brainrender import Animation, Scene, settings
from pathlib import Path

settings.SHOW_AXES = False

scene = Scene(atlas_name="allen_mouse_10um")

regions = (
    "CTX",
    "HPF",
    "STR",
    "CB",
    "MB",
    "TH",
    "HY",
)
scene.add_brain_region(*regions, silhouette=True)


def slc(scene, framen, totframes):
    # Get new slicing plane
    fact = framen / totframes
    shape_um = scene.atlas.shape_um
    # Multiply by fact to move the plane, add buffer to go past the brain
    point = [(shape_um[0] + 500) * fact, shape_um[1] // 2, shape_um[2] // 2]
    plane = scene.atlas.get_plane(pos=point, norm=(1, 0, 0))

    scene.slice(plane)


anim = Animation(
    scene, Path.cwd(), "brainrender_animation_callback", size=None
)

# Specify camera pos and zoom at some key frames`
anim.add_keyframe(0, camera="frontal", zoom=1, callback=slc)

# Make videos
anim.make_video(duration=5, fps=10, fix_camera=True)

Saving video in /ceph/behrens/georgy_antonov/Explore_exploit/experiment/code

📽  Video file brainrender_animation_callback is open... 

Output()

Saving video

Saved video at: /ceph/behrens/georgy_antonov/Explore_exploit/experiment/code/brainrender_animation_callback.mp4

'/ceph/behrens/georgy_antonov/Explore_exploit/experiment/code/brainrender_animation_callback.mp4'

In [5]:
## testing brainrender

import brainrender as br

scene = br.Scene(title="Silicon Probe Visualization", 
                 atlas_name = "allen_mouse_10um")

# Visualise the probe target regions
cp = scene.add_brain_region("CP", alpha=0.15)
rsp = scene.add_brain_region("RSP", alpha=0.15)

# Add probes to the scene.
# Each .npy file should contain a numpy array with the coordinates of each


# render
scene.export("../results/brainrender/test_brain_regions.html")

2025-08-11 15:33:37.555 ( 167.437s) [    7F7575576740]vtkXOpenGLRenderWindow.:1458  WARN| bad X server connection. DISPLAY=
[2025-08-11 15:34:54,103] INFO in helpers: Converting int64 array to int32 for JS compatibility.
[2025-08-11 15:34:54,181] INFO in helpers: Converting int64 array to int32 for JS compatibility.
[2025-08-11 15:34:54,211] INFO in helpers: Converting int64 array to int32 for JS compatibility.
[2025-08-11 15:34:54,220] INFO in helpers: Converting int64 array to int32 for JS compatibility.
[2025-08-11 15:34:54,222] INFO in helpers: Converting int64 array to int32 for JS compatibility.
[2025-08-11 15:34:54,224] INFO in helpers: Converting int64 array to int32 for JS compatibility.


The brainrender scene has been exported for web. The results are saved at 
../results/brainrender/test_brain_regions.html

'../results/brainrender/test_brain_regions.html'

In [9]:
"""
This example shows how to create an animated video by specifying
the camera parameters at a number of key frames
"""

from pathlib import Path

from brainrender import Animation, Scene

# Create a brainrender scene
scene = Scene(title="brain regions",atlas_name = "allen_mouse_10um", inset=False)

# Add brain regions
scene.add_brain_region("TH")

anim = Animation(scene, Path.cwd(), "brainrender_animation")

# Specify camera position and zoom at some key frames
# each key frame defines the scene's state after n seconds have passed
anim.add_keyframe(0, camera="top", zoom=1)
anim.add_keyframe(1.5, camera="sagittal", zoom=0.95)
anim.add_keyframe(3, camera="frontal", zoom=1)
anim.add_keyframe(4, camera="frontal", zoom=1.2)

# Make videos
anim.make_video(duration=5, fps=15, resetcam=True)

Saving video in /ceph/behrens/georgy_antonov/Explore_exploit/experiment/code

📽  Video file brainrender_animation is open... 

Output()

Saving video

Saved video at: /ceph/behrens/georgy_antonov/Explore_exploit/experiment/code/brainrender_animation.mp4

'/ceph/behrens/georgy_antonov/Explore_exploit/experiment/code/brainrender_animation.mp4'